In [1]:
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import pandas as pd
import numpy as np
import panel as pn

from sklearn import datasets, linear_model
from sklearn.metrics import r2_score

In [2]:
pn.extension('vega')

In [3]:
data = datasets.load_diabetes()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['Target'] = data.target
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,Target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [4]:
X = df.drop('Target', axis=1)
y = df.Target

In [5]:
lr = linear_model.LinearRegression()
lr.fit(X, y)

y_pred = lr.predict(X)
r2 = r2_score(y, y_pred)
r2

0.5177484222203499

In [6]:
def get_regression_line(df):
    lr = linear_model.LinearRegression()
    lr.fit(X, y)
    coef = lr.coef_
    intercept = lr.intercept_
    
    scatter_plot = alt.Chart(df).mark_circle().encode(
        x='bmi',
        y='Target',
        tooltip=['bmi', 'Target'])
    
    line_plot = alt.Chart(pd.DataFrame({'bmi': [df['bmi'].min(), 
                                    df['bmi'].max()], 'Target': [df['bmi'].min() * coef[2] + intercept, 
                                    df['bmi'].max() * coef[2] + intercept]})).mark_line(color='red').encode(
        x='bmi', y='Target')
    return scatter_plot + line_plot

In [7]:
get_regression_line(df)

alt.LayerChart(...)

In [8]:
def create_box_plot(df):
    return alt.Chart(df).mark_boxplot().encode(
    x=alt.X('key:N', title='Feature'),
    y=alt.Y('value:Q', title='Value'),).transform_fold(list(df.columns[:-1]),
                                                    as_=['key','value']).interactive()

In [9]:
create_box_plot(df)

alt.Chart(...)

In [10]:
def create_scatter_plot(df):
    return alt.Chart(df).mark_circle().encode(x='bmi', y='Target', tooltip=['bmi','Target'])

def scatter_plot(x_feature, y_feature):
    chart = alt.Chart(df).mark_circle(size=60).encode(
    x=x_feature, y=y_feature, color='Target',
    tooltip=[x_feature, y_feature, 'Target']).properties(width=600, height=400)
    return chart

In [11]:
x_select = pn.widgets.Select(options=list(df.columns), name='X-axis')
y_select = pn.widgets.Select(options=list(df.columns), name='Y-axis')

@pn.depends(x_select.param.value, y_select.param.value)
def update_plot(x_feature, y_feature):
    return scatter_plot(x_feature, y_feature)

In [12]:
dashboard = pn.Column(
    pn.pane.Markdown('# Diabetes Regression Analysis Dashboard'),
    pn.Row(get_regression_line(df), create_box_plot(df)),
    pn.Column(x_select, y_select), update_plot)

In [13]:
dashboard

Column
    [0] Markdown(str)
    [1] Row
        [0] Vega(LayerChart, selection=Selection)
        [1] Vega(Chart, selection=Selection)
    [2] Column
        [0] Select(name='X-axis', options=['age', 'sex', ...], value='age')
        [1] Select(name='Y-axis', options=['age', 'sex', ...], value='age')
    [3] ParamFunction(function, _pane=Vega, defer_load=False)

In [22]:
# Create the Altair charts
boxplot = alt.Chart(df).mark_boxplot().encode(
    x=alt.X('key:N', title='Feature'),
    y=alt.Y('value:Q', title='Value'),
).transform_fold(
    list(df.columns[:-1]),
    as_=['key', 'value']
).interactive()

scatterplot = alt.Chart(df).mark_circle().encode(
    x=alt.X(alt.repeat("column"), type='quantitative'),
    y=alt.Y('target:Q'),
    tooltip=list(df.columns)
).properties(width=150, height=150).repeat(column=list(df.columns[:-1])).interactive()

linear_regression = alt.Chart(pd.DataFrame({'y': y, 'y_pred': y_pred})).mark_point().encode(
    x=alt.X('y:Q', title='True Target Values'),
    y=alt.Y('y_pred:Q', title='Predicted Target Values'),
    tooltip=['y', 'y_pred']
).interactive()



In [25]:
dashboard2 = pn.Column(
    pn.pane.Markdown('# Diabetes Regression Dashboard'),
    pn.Row(boxplot, scatterplot),
    pn.Row(linear_regression),
    pn.Row(r2),)

In [26]:
dashboard2.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] Vega(Chart, selection=Selection)
        [1] Vega(RepeatChart, selection=Selection)
    [2] Row
        [0] Vega(Chart, selection=Selection)
    [3] Row
        [0] Str(float64)